In [1]:
import requests
import urllib.request
import os, time
from bs4 import BeautifulSoup
import urllib

In [2]:
def retrieve_categories(base_url, category_head):
    
    ### Get all Microdata categories ###
    categories_url = base_url+category_head
    categories_response = requests.get(categories_url)

    categories_soup = BeautifulSoup(categories_response.text)
    categories_browse = categories_soup.findAll('a')
    
    categories_url_list = []
    categories_list = []
    for item in categories_browse:
        try:
            if category_head in item['href']:
                categories_url_list.append(base_url+item['href'])
                categories_list.append(item['href'].split('/')[-1])
        except KeyError:
            pass
    
    return categories_url_list, categories_list

In [3]:
def retrieve_variables(variables_url, variable_base_url):

    ### Crawl all variables names on the category web page ###

    variables_response = requests.get(variables_url)

    variables_soup = BeautifulSoup(variables_response.text)
    variables_browse = variables_soup.findAll('a')

    ### List all variables into a list ###
    variables_list = []
    for item in variables_browse:
        try:
            variable = item["data-id"][:-2]
            variables_list.append(variable)
        except KeyError:
            pass
            
    variable_url_list = []
    for item in variables_list:
        variable_url_list.append(variable_base_url+item)
            
    return variable_url_list, variables_list

In [4]:
def download_page(variable_url_list, base_url):
### Obtain Metadata phd file ###
    time.sleep(1)
    download_pages = []

    for each_variable_url in variable_url_list:

        each_variable_response = requests.get(each_variable_url)

        each_variable_soup = BeautifulSoup(each_variable_response.text)
        each_variable_browse = each_variable_soup.findAll('a')

        for item in each_variable_browse:
            try:
                if ".pdf" in item["href"]:
                    download_pages.append(base_url+item["href"])
            except KeyError:
                pass
            except:
                print("This is not a KeyError!")
                
    return download_pages


In [5]:
### Download PDF files ###
def download_pdf(download_pages, each_category):
    save_path_base = os.getcwd() + "/MicrodataFiles/%s/" %(each_category)
    
    # Check if folder exists #
    create_new_folder(save_path_base)
    
    
    for item in range(0, len(download_pages)):
        extracted_file_name = download_pages[item].split('/')[-1]
        file_name_to_save = save_path_base + extracted_file_name.split('?la=nl-nl')[0]
        
        ifExist = check_folder_exist(file_name_to_save)
        if ifExist == "is_a_file":
            print("%s has been downloaded!" %extracted_file_name.split('?la=nl-nl')[0])
        else:
            try:
                
                urllib.request.urlretrieve(download_pages[item], file_name_to_save)
                time.sleep(1)
            except:
                print("%s is NOT downloaded! Please try it again later." %extracted_file_name.split('?la=nl-nl')[0])

In [6]:
def check_folder_exist(file_path):
    
    ifExist = False
    # If this file object exist.
    if(os.path.exists(file_path)):
        
        # If this is a file.
        if(os.path.isfile(file_path)):
            ifExist = "is_a_file"

        # This is a directory.    
        else:
            ifExist = True
            
    else:
        ifExist = False
        
    return ifExist

def create_new_folder(file_path):
    if(not check_folder_exist(file_path)):
        os.mkdir(file_path)
        print(file_path + " has been created. ")

In [1]:
# def main(): 
base_url = "https://www.cbs.nl"
category_head = "/nl-nl/onze-diensten/maatwerk-en-microdata/microdata-zelf-onderzoek-doen/catalogus-microdata/"
variable_base_url = "https://www.cbs.nl/nl-nl/onze-diensten/maatwerk-en-microdata/microdata-zelf-onderzoek-doen/microdatabestanden/"

# Retrieve categories #
categories_url_list, categories_list = retrieve_categories(base_url, category_head)
print("There are %s categories" %str(len(categories_url_list)))

overview = []
# Retrieve variables in each category #
for category_item in range(0, len(categories_url_list)):

    variable_url_list, variables_list = retrieve_variables(categories_url_list[category_item], variable_base_url)
    print("In category (%s), there are %s variables" %(categories_list[category_item], str(len(variable_url_list))))
    overview.append(variables_list)
    download_pages = download_page(variable_url_list, base_url)
    
    download_pdf(download_pages, categories_list[category_item])

    print("****** Retrieval of %s is done! ******" %(categories_list[category_item]))

In [15]:
import pandas as pd
overview_dataFrame = pd.DataFrame(overview).transpose()
overview_dataFrame.columns = categories_list

In [16]:
overview_dataFrame.to_csv("overview.csv")